In [ ]:
import torch
print(torch)

In [5]:
cd /content/drive/MyDrive/ai_project/AI-Practical-Research-Project-2022-2

/content/drive/MyDrive/ai_project/AI-Practical-Research-Project-2022-2


In [7]:
!git add .
!git commit -m "fix"
!git push


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@5dda795a0199.(none)')
Everything up-to-date
